In [1]:
!nvidia-smi

Sun Jan 26 02:05:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
if (device_name != b'Tesla T4') and (device_name != b'Tesla P100-PCIE-16GB'):
  raise Exception("""
    Unfortunately this instance does not have a T4 or P100 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4 or P100. Resetting the instance.
If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print(f'Yes, you got the right kind of GPU to work and it is a ({device_name}) GPU.')

Yes, you got the right kind of GPU to work and it is a (b'Tesla T4') GPU.


In [6]:
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

--2020-01-26 02:07:27--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rapidsai/notebooks-contrib/raw/master/utils/rapids-colab.sh [following]
--2020-01-26 02:07:28--  https://github.com/rapidsai/notebooks-contrib/raw/master/utils/rapids-colab.sh
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/rapids-colab.sh [following]
--2020-01-26 02:07:28--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.co

# **Principal Componenet Analysis (PCA)**
The PCA algorithm is a dimensionality reduction algorithm which works really well for datasets which have correlated columns. It combines the features of X in linear combination such that the new components capture the most information of the data. 
The PCA model is implemented in the cuML library and can accept the following parameters: 
1. svd_solver: selects the type of algorithm used: Jacobi or full (default = full)
2. n_components: the number of top K vectors to be present in the output (default = 1)
3. random_state: select a random state if the results should be reproducible across multiple runs (default = None)
4. copy: if 'True' then it copies the data and removes mean from it else the data will be overwritten with its mean centered version (default = True)
5. whiten: if True, de-correlates the components (default = False)
6. tol: if the svd_solver = 'Jacobi' then this variable is used to set the tolerance (default = 1e-7)
7. iterated_power: if the svd_solver = 'Jacobi' then this variable decides the number of iterations (default = 15)

The cuml implementation of the PCA model has the following functions that one can run:
1. Fit: it fits the model with the dataset
2. Fit_transform: fits the PCA model with the dataset and performs dimensionality reduction on it
3. Inverse_transform: returns the original dataset when the transformed dataset is passed as the input
4. Transform: performs dimensionality reduction on the dataset
5. Get_params: returns the value of the parameters of the PCA model
6. Set_params: allows the user to set the value of the parameters of the PCA model

The model accepts only numpy arrays or cudf dataframes as the input. In order to convert your dataset to cudf format please read the cudf documentation on https://rapidsai.github.io/projects/cudf/en/latest/. For additional information on the PCA model please refer to the documentation on https://rapidsai.github.io/projects/cuml/en/latest/index.html

In [0]:
import os

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA as skPCA

import cudf
from cuml import PCA as cumlPCA

In [0]:
# check if the mortgage dataset is present and then extract the data from it, else do not run 
import gzip
def load_data(nrows, ncols, cached = 'mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        # throws FileNotFoundError error if mortgage dataset is not present
        raise FileNotFoundError('Please download the required dataset or check the path')
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [0]:
# this function checks if the results obtained from two different methods (sklearn and cuml) are the equal
from sklearn.metrics import mean_squared_error
def array_equal(a, b, threshold=2e-3, with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

# the function converts a variable from ndarray or dataframe format to numpy array
def to_nparray(x):
    if isinstance(x, np.ndarray) or isinstance(x, pd.DataFrame):
        return np.array(x)
    elif isinstance(x, np.float64):
        return np.array([x])
    elif isinstance(x, cudf.DataFrame) or isinstance(x, cudf.Series):
        return x.to_pandas().values
    return x    

##  **Loading Data**

In [11]:
from google.colab import files
uploaded = files.upload()

Saving mortgage.npy.gz to mortgage.npy.gz


In [18]:
%%time
# nrows = number of samples
# ncols = number of features of each sample

nrows = 2**15
nrows = int(nrows * 1.5)
ncols = 400

X = load_data(nrows,ncols)
print('data', X.shape)

use mortgage data
data (49152, 400)
CPU times: user 5.36 s, sys: 695 ms, total: 6.05 s
Wall time: 6.06 s


In [13]:
X

,fea0,fea1,fea2,fea3,fea4,fea5,fea6,fea7,fea8,fea9,fea10,fea11,fea12,fea13,fea14,fea15,fea16,fea17,fea18,fea19,fea20,fea21,fea22,fea23,fea24,fea25,fea26,fea27,fea28,fea29,fea30,fea31,fea32,fea33,fea34,fea35,fea36,fea37,fea38,fea39,...,fea360,fea361,fea362,fea363,fea364,fea365,fea366,fea367,fea368,fea369,fea370,fea371,fea372,fea373,fea374,fea375,fea376,fea377,fea378,fea379,fea380,fea381,fea382,fea383,fea384,fea385,fea386,fea387,fea388,fea389,fea390,fea391,fea392,fea393,fea394,fea395,fea396,fea397,fea398,fea399
0,0.486111,0.181862,0.081081,0.378906,0.452778,0.709690,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.652778,0.426570,0.581081,0.513672,0.580556,0.625251,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.666667,0.407274,0.045045,0.746094,0.972222,0.000000,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.694444,0.268575,0.103604,0.720703,0.938889,0.765179,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.652778,0.386920,0.094595,0.724609,0.944444,0.290712,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,0.513889,0.105926,0.301802,0.283203,0.316667,0.000000,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49148,0.500000,0.189171,0.337838,0.384766,0.461111,0.277845,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49149,0.708333,0.315027,0.090090,0.726562,0.936111,0.230398,0.0,0.044486,0.059951,0.061698,0.716319,0.083032,0.231624,0.113612,0.212412,0.00678,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

##  **Model Parameter**

In [0]:
# set parameters for the PCA model
n_components = 10
whiten = False
random_state = 42
svd_solver = "full"

## **Scikit-Learn Implementation**

In [20]:
%%time
# use the sklearn PCA on the dataset
pca_sk = skPCA(n_components = n_components,
               svd_solver = svd_solver, 
               whiten = whiten, 
               random_state = random_state)

# creates an embedding
result_sk_pca = pca_sk.fit_transform(X)

CPU times: user 2.94 s, sys: 361 ms, total: 3.3 s
Wall time: 1.86 s


## **cuML Implementation**

In [21]:
%%time
# convert the pandas dataframe to cudf dataframe
X = cudf.DataFrame.from_pandas(X)

CPU times: user 358 ms, sys: 8.82 ms, total: 367 ms
Wall time: 374 ms


In [22]:
%%time
# use the cuml PCA model on the dataset
pca_cuml = cumlPCA(n_components = n_components,
                   svd_solver = svd_solver, 
                   whiten = whiten, 
                   random_state = random_state)
# obtain the embedding of the model
result_cuml_pca = pca_cuml.fit_transform(X)

CPU times: user 1.52 s, sys: 365 ms, total: 1.89 s
Wall time: 2.02 s


In [23]:
# calculate the attributes of the two models and compare them
for attr in ['singular_values_',
             'components_',
             'explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_sk, attr), getattr(pca_cuml, attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn          singular_values_ equal
compare pca: cuml vs sklearn               components_ equal
compare pca: cuml vs sklearn       explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal


In [24]:
# compare the results of the two models
passed = array_equal(result_sk_pca, result_cuml_pca)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results equal
